In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import alphabase.psm_reader
import alpharaw.utils.ms_path_utils
from alpharaw.viz.psm_plot import PSM_Plot
from alpharaw.viz.xic_plot import XIC_Plot
from alpharaw.viz.df_utils import (
    make_psm_plot_df_for_peptide,
    make_xic_plot_df_for_peptide
)
from alpharaw.wrappers.alphatims_wrapper import AlphaTimsWrapper
from peptdeep.pretrained_models import ModelManager

import alphaviz.alphax_utils as xutils

psm_plotter = PSM_Plot()
xic_plotter = XIC_Plot()
model_mgr = ModelManager(mask_modloss=False)

raw_file = "/Users/wenfengzeng/data/ap_msdata/HeLa_500ng.raw.hdf"
dda = True

psm_file = "/Users/wenfengzeng/data/ap_msdata/msms.txt"
psm_file_type = "maxquant"
use_peptdeep = True

raw_name = alpharaw.utils.ms_path_utils.get_raw_name(raw_file)
msdata = xutils.get_msdata(raw_file)
tims_data = AlphaTimsWrapper(msdata, dda=dda)

psm_df = alphabase.psm_reader.psm_reader_provider.get_reader(
    psm_file_type
).import_file(psm_file)
psm_df = psm_df.query(f"raw_name=='{raw_name}'").reset_index(drop=True)

key_cols = ["sequence","mods","mod_sites","charge","rt"]
psm_df = pd.concat(
    [
        psm_df[key_cols], 
        psm_df[[col for col in psm_df.columns if col not in key_cols]]
    ],
    axis=1
)
psm_id = 100
        
plot_psm = psm_df.loc[psm_id:psm_id]
plot_psm

,sequence,mods,mod_sites,charge,rt,scan_num,raw_name,precursor_mz,score,proteins,genes,decoy,spec_idx,nAA,rt_norm
100,SLLATMK,,,2,44.227,48002,HeLa_500ng,382.22276,86.866,P15924,DSP,0,48001,7,0.369575


In [5]:

plot_df = make_xic_plot_df_for_peptide(
    sequence=plot_psm.sequence.values[0],
    mods=plot_psm.mods.values[0],
    mod_sites=plot_psm.mod_sites.values[0],
    charge=plot_psm.charge.values[0],
    rt_sec=plot_psm.rt.values[0]*60,
    ms_level=1 if dda else 2,
    include_precursor_isotopes=True if dda else False,
)

fig = xic_plotter.plot(
    tims_data=tims_data, 
    query_df=plot_df,
    title=plot_df.modified_sequence.values[0], 
)
fig.show()

In [4]:
if use_peptdeep:
    pred_inten_df = model_mgr.predict_ms2(plot_psm)
else:
    pred_inten_df = None

spec_idx = plot_psm.spec_idx.values[0]
peak_mzs, peak_intens = msdata.get_peaks(spec_idx)

plot_df = make_psm_plot_df_for_peptide(
    peak_mzs, peak_intens,
    sequence=plot_psm.sequence.values[0],
    mods=plot_psm.mods.values[0],
    mod_sites=plot_psm.mod_sites.values[0],
    charge=plot_psm.charge.values[0],
    fragment_intensity_df=pred_inten_df,
)

fig = psm_plotter.plot(
    plot_df, 
    plot_psm.sequence.values[0], 
    plot_df.modified_sequence.values[0], 
    plot_unmatched_peaks=True
)
fig.show()

2024-01-20 21:23:51> Predicting MS2 ...


100%|██████████| 1/1 [00:00<00:00, 43.75it/s]
